In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
import tvm
from tvm import relay
from tvm.relay.dataflow_pattern import rewrite
from tvm.relay.dataflow_pattern import *
import numpy as np
from tvm.relay import transform
from tvm.relay.testing import run_opt_pass
from tvm.relay import transform, build_module
from tvm.relay import testing
import tvm.testing
# from SlicingMachine import TVMSlicer
from mozer.slicer.SlicingMachine import TVMSlicer
from mozer.slicer.Quantize import quantize
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tvm
import tvm.relay as relay
from tvm.contrib import graph_executor 
import numpy as np
import json
import pygraphviz as pgv
from argparse import ArgumentParser
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *
from tvm import rpc
from tvm.autotvm.measure.measure_methods import set_cuda_target_arch
from tensorflow import keras
from tvm.contrib.download import download_testdata
import subprocess
from tvm.contrib import relay_viz
from tvm.relay import build_module

/home/jd/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
def graph_split(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            # if is_free_var:
            #     need_update = True
            #     new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
            #     new_input_idx += 1
            # else:
            new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    if (
                        split_conf
                        and split_operator_name in operator_index_map
                        and operator_index_map[split_operator_name] >= split_operator_index
                    ):
                        # Do graph splitting.
                        split_conf.pop(0)
                        snode_dep.append({"nodes": {}, "ref_nodes": {}})
                        ann = _recursion(
                            anf.body,
                            pipeline_mods,
                            split_conf,
                            constant_expr,
                        )
                        snode_dep.pop()
                        dep_vars = get_dep_var(snode_dep)
                        # When the nodes of the current subgraph are the depedency node of another
                        # subgraph, we need to set them as the output of current subgraph.
                        body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                        # when the operator of current subgraph uses previous subgraph constant
                        # as the argument of a "relay.expr.call", such constant may become a free
                        # varaible if the constant does not exist in the current subgraph.
                        # merge the previous constant with current subgraph to avoid such issue.
                        if constant_expr:
                            ann = merge_constant_expr(constant_expr, ann)
                        ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                        mod = tvm.IRModule.from_expr(ann)
                        pipeline_mods.insert(0, mod)
                        # Return the last node of the current subgraph.
                        return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf
    
    # def getting_inputs(mod):
    #     name_hints = []
    #     for param in mod['main'].params:
    #         name_hints.append(param.name_hint)
    #     return name_hints

    # def setting_outputs(mod, name_hints):
    #     # Get last element in snode_dep : current node's dependency
    #     # If function -> decouple
    #     if isinstance(anf, tvm.relay.Function):
    #         return tvm.relay.Function(
    #             anf.params,
    #             _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
    #             anf.ret_type,
    #             anf.type_params,
    #             anf.attrs,
    #         )
    #     # Function of Let
    #     if isinstance(anf, tvm.relay.expr.Let):
    #         value = anf.value
    #         # record the constant expr to make sure all sugraphs can find correct constant.
    #         if isinstance(value, tvm.relay.expr.Constant):
    #             # cosntant_expr is initally None
    #             if not constant_expr:
    #                 constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
    #             else:
    #                 constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
    #         if isinstance(value, tvm.relay.expr.Call):
    #             new_args = []
    #             # build current var list
    #             cur_node_dep["nodes"][anf.var] = 0
    #             # Get the dependency information of the nodes.
    #             value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
    #             if isinstance(value.op, tvm.ir.Op):
    #                 if value.op.name in operator_index_map:
    #                     operator_index_map[value.op.name] += 1
    #                 else:
    #                     operator_index_map[value.op.name] = 0
    #                 split_operator_name = split_conf[0]["op_name"] if split_conf else ""
    #                 split_operator_index = split_conf[0]["op_index"] if split_conf else ""
    #                 # if a operator name and repeating count in the network match with the values
    #                 # of the 'split configuration', then this place is where we should do the
    #                 # graph splitting.
    #                 if (
    #                     split_conf
    #                     and split_operator_name in operator_index_map
    #                     and operator_index_map[split_operator_name] >= split_operator_index
    #                 ):
    #                     # Do graph splitting.
    #                     split_conf.pop(0)
    #                     snode_dep.append({"nodes": {}, "ref_nodes": {}})
    #                     ann = _recursion(
    #                         anf.body,
    #                         pipeline_mods,
    #                         split_conf,
    #                         constant_expr,
    #                     )
    #                     snode_dep.pop()
    #                     dep_vars = get_dep_var(snode_dep)
    #                     # When the nodes of the current subgraph are the depedency node of another
    #                     # subgraph, we need to set them as the output of current subgraph.
    #                     body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
    #                     # when the operator of current subgraph uses previous subgraph constant
    #                     # as the argument of a "relay.expr.call", such constant may become a free
    #                     # varaible if the constant does not exist in the current subgraph.
    #                     # merge the previous constant with current subgraph to avoid such issue.
    #                     if constant_expr:
    #                         ann = merge_constant_expr(constant_expr, ann)
    #                     ann = run_opt_pass(ann, transform.ToGraphNormalForm())
    #                     mod = tvm.IRModule.from_expr(ann)
    #                     pipeline_mods.insert(0, mod)
    #                     # Return the last node of the current subgraph.
    #                     return tvm.relay.expr.Let(anf.var, value, body)
    #         return tvm.relay.expr.Let(
    #             anf.var,
    #             value,
    #             _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
    #         )
    #     # Or End
    #     else:
    #         return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, mod)

    return pipeline_mods


In [2]:
def graph_split2(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            if is_free_var:
                need_update = True
                new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
                new_input_idx += 1
            else:
                new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    if (
                        split_conf
                        and split_operator_name in operator_index_map
                        and operator_index_map[split_operator_name] >= split_operator_index
                    ):
                        # Do graph splitting.
                        split_conf.pop(0)
                        snode_dep.append({"nodes": {}, "ref_nodes": {}})
                        ann = _recursion(
                            anf.body,
                            pipeline_mods,
                            split_conf,
                            constant_expr,
                        )
                        snode_dep.pop()
                        dep_vars = get_dep_var(snode_dep)
                        # When the nodes of the current subgraph are the depedency node of another
                        # subgraph, we need to set them as the output of current subgraph.
                        body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                        # when the operator of current subgraph uses previous subgraph constant
                        # as the argument of a "relay.expr.call", such constant may become a free
                        # varaible if the constant does not exist in the current subgraph.
                        # merge the previous constant with current subgraph to avoid such issue.
                        if constant_expr:
                            ann = merge_constant_expr(constant_expr, ann)
                        # ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                        # mod = tvm.IRModule.from_expr(ann)
                        pipeline_mods.insert(0, ann)
                        # Return the last node of the current subgraph.
                        return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf
    
    def getting_inputs(mod):
        name_hints = []
        ann = run_opt_pass(mod.body, transform.ToGraphNormalForm())
        mod = tvm.IRModule.from_expr(ann)
        for param in mod['main'].params:
            name_hints.append(param.name_hint)
        return name_hints

    def setting_outputs(anf, name_hints, outputs):
        # Get last element in snode_dep : current node's dependency
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                setting_outputs(anf.body, name_hints, outputs),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # print(anf.var.name_hint)
            if anf.var.name_hint in name_hints:
                outputs.append(anf)
            # value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                setting_outputs(anf.body, name_hints, outputs),
            )
        # Or End
        else:
            # new_map = 
            return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    # ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    # mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, ann.body)
    inputs = []
    for mod in pipeline_mods:
        inputs.extend(getting_inputs(mod))
    print(inputs)
    total_outputs = []
    for mod in pipeline_mods:
        outputs = []
        setting_outputs(mod, inputs, outputs)
        total_outputs.append(outputs)
    return pipeline_mods, total_outputs


In [3]:
def get_inputs(expr, split_conf, params=None):
    """Splitting the graph into a list of subgraphs"""

    def get_dep_var(sub_var_dep):
        return [var for var in sub_var_dep[len(sub_var_dep) - 1]["ref_nodes"]]

    def parse_dependency(value, snode_dep, new_input_idx):
        new_args = []
        need_update = False
        for var in value.args:
            is_free_var = False
            for dep in snode_dep[:-1]:
                if var in dep["nodes"]:
                    # Mark the previous subgraph node as a dependency.
                    dep["nodes"][var] += 1
                    dep["ref_nodes"][var] = dep["nodes"][var]
                    # The var of this call is a free_var
                    is_free_var = True
            # if the var of this call is a free_var, recreate it and give it a fixed input name.
            if is_free_var:
                need_update = True
                new_args.append(relay.var(f"data_n_{new_input_idx}", var.checked_type))
                new_input_idx += 1
            else:
                new_args.append(var)
        # if the 'tvm.relay.expr.Call' has a free_var, recreate it with new name as 'data_n_*'.
        if need_update:
            value = tvm.relay.expr.Call(
                value.op, new_args, value.attrs, value.type_args, value.span
            )
        return value, snode_dep, new_input_idx

    def merge_constant_expr(constant_expr, expr):
        # merge constant express with a express
        if not isinstance(constant_expr.body, tvm.relay.expr.Let):
            return tvm.relay.expr.Let(constant_expr.var, constant_expr.value, expr)

        return tvm.relay.expr.Let(
            constant_expr.var, constant_expr.value, merge_constant_expr(constant_expr.body, expr)
        )

    def _recursion(anf, pipeline_mods, split_conf, constant_expr):
        # Enumurate all operators of compute graph, then split the compute graph into a group of
        # subgraph.
        nonlocal operator_index_map
        nonlocal new_input_idx
        nonlocal snode_dep
        # Get last element in snode_dep : current node's dependency
        cur_node_dep = snode_dep[len(snode_dep) - 1]
        # If function -> decouple
        if isinstance(anf, tvm.relay.Function):
            return tvm.relay.Function(
                anf.params,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
                anf.ret_type,
                anf.type_params,
                anf.attrs,
            )
        # Function of Let
        if isinstance(anf, tvm.relay.expr.Let):
            value = anf.value
            # record the constant expr to make sure all sugraphs can find correct constant.
            if isinstance(value, tvm.relay.expr.Constant):
                # cosntant_expr is initally None
                if not constant_expr:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, anf.var)
                else:
                    constant_expr = tvm.relay.expr.Let(anf.var, value, constant_expr)
            if isinstance(value, tvm.relay.expr.Call):
                print(anf.var.name_hint)
                print(anf.var)
                new_args = []
                # build current var list
                cur_node_dep["nodes"][anf.var] = 0
                # Get the dependency information of the nodes.
                value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
                if isinstance(value.op, tvm.ir.Op):
                    if value.op.name in operator_index_map:
                        operator_index_map[value.op.name] += 1
                    else:
                        operator_index_map[value.op.name] = 0
                    # split_operator_name = split_conf[0]["op_name"] if split_conf else ""
                    # split_operator_index = split_conf[0]["op_index"] if split_conf else ""
                    # if a operator name and repeating count in the network match with the values
                    # of the 'split configuration', then this place is where we should do the
                    # graph splitting.
                    # if (
                    #     split_conf
                    #     and split_operator_name in operator_index_map
                    #     and operator_index_map[split_operator_name] >= split_operator_index
                    # ):
                    #     # Do graph splitting.
                    #     split_conf.pop(0)
                    #     snode_dep.append({"nodes": {}, "ref_nodes": {}})
                    #     ann = _recursion(
                    #         anf.body,
                    #         pipeline_mods,
                    #         split_conf,
                    #         constant_expr,
                    #     )
                    #     snode_dep.pop()
                    #     dep_vars = get_dep_var(snode_dep)
                    #     # When the nodes of the current subgraph are the depedency node of another
                    #     # subgraph, we need to set them as the output of current subgraph.
                    #     body = relay.Tuple(dep_vars) if len(dep_vars) > 1 else anf.var
                    #     # when the operator of current subgraph uses previous subgraph constant
                    #     # as the argument of a "relay.expr.call", such constant may become a free
                    #     # varaible if the constant does not exist in the current subgraph.
                    #     # merge the previous constant with current subgraph to avoid such issue.
                    #     if constant_expr:
                    #         ann = merge_constant_expr(constant_expr, ann)
                    #     ann = run_opt_pass(ann, transform.ToGraphNormalForm())
                    #     mod = tvm.IRModule.from_expr(ann)
                    #     pipeline_mods.insert(0, mod)
                    #     # Return the last node of the current subgraph.
                        # return tvm.relay.expr.Let(anf.var, value, body)
            return tvm.relay.expr.Let(
                anf.var,
                value,
                _recursion(anf.body, pipeline_mods, split_conf, constant_expr),
            )
        # Or End
        else:
            return anf

    snode_dep = [{"nodes": {}, "ref_nodes": {}}]
    pipeline_mods = []
    operator_index_map = {}
    # Used to tracking new input which caused by graph splitting.
    new_input_idx = 0
    constant_expr = None
    subgraph_split_conf = split_conf.copy()
    # Binding the parameters.
    if params:
        expr = build_module.bind_params_by_name(expr, params)
    anf = run_opt_pass(expr, transform.ToANormalForm())
    anf = run_opt_pass(anf, transform.InferType())
    ann = _recursion(
        anf,
        pipeline_mods,
        subgraph_split_conf,
        constant_expr,
    )
    ann = run_opt_pass(ann.body, transform.ToGraphNormalForm())
    mod = tvm.IRModule.from_expr(ann)
    pipeline_mods.insert(0, mod)
    return pipeline_mods


In [32]:
def setting_outputs(anf, outputs):
    # Get last element in snode_dep : current node's dependency
    # If function -> decouple
    if isinstance(anf, tvm.relay.Function):
        return tvm.relay.Function(
            anf.params,
            setting_outputs(anf.body, outputs),
            anf.ret_type,
            anf.type_params,
            anf.attrs,
        )
    # Function of Let
    if isinstance(anf, tvm.relay.expr.Let):
        value = anf.value
        # print(anf.var.name_hint)
        # if anf.var.name_hint in name_hints:
            # outputs.append(anf)
        # value, snode_dep, new_input_idx = parse_dependency(value, snode_dep, new_input_idx)
        return tvm.relay.expr.Let(
            anf.var,
            value,
            setting_outputs(anf.body, outputs),
        )
    # Or End
    else:
        new_outputs = []
        for o in outputs:
            new_outputs.append(o.var)
        # for o in outputs:
        #     new_outputs.append(tvm.relay.expr.Let(
        #             o.var,
        #             o.value,
        #             o.body,
        #     ))
        new_map = tvm.relay.expr.Tuple(new_outputs)
        return new_map

In [34]:
class SetOutputCallback(DFPatternCallback):
    def __init__(self, name_hints=[],require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.pattern = wildcard()

    def callback(self, pre, post, node_map):
        try:
            print(type(pre))
            print(pre.name_hint)
        except:
            pass
        return pre 

In [3]:
base_path = "./simple_model.h5"
input_data = np.random.normal(0,1,(1,256,256,3)).astype(np.float32)
model_keras = tf.keras.models.load_model(base_path)
# tvm result
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)


In [6]:
out = quantize(mod, 2)
split_config = [{"op_name": "annotation.stop_fusion", "op_index": 0}]
# print(mod['main'])
subgraphs, outputs = graph_split2(out, split_config, params)

2
2
[CallNode(Op(concatenate), [Tuple([CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d_transpose), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.bias_add), [CallNode(Op(nn.conv2d), [Var(input_1, ty=TensorType([1, 3, 256, 256], float32)), Var(_param_1, ty=TensorType([16, 3, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x37677f8), []), Var(_param_2, ty=TensorType([16], float32))], relay.attrs.BiasAddAttrs(0x311e1cc8), [])], relay.attrs.MaxPool2DAttrs(0x3767468), []), Var(_param_3, ty=TensorType([32, 16, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x311d4118), []), Var(_param_4, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x6183038), [])], relay.attrs.MaxPool2DAttrs(0x311c5df8), []), Var(_param_5, ty=TensorType([64, 32, 3, 3], float32))], relay.attrs.Conv2DAttrs(0x311b22e8), []), Var(_param_6, ty=TensorType([64], float3

In [15]:
[o.var.name_hint for o in outputs[1]]
outputs[0][0].var.name_hint

['x_0']

In [33]:
modmod = setting_outputs(subgraphs[0], outputs[0][1:])

In [36]:
ann = run_opt_pass(modmod, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']


In [8]:
print([o.var for o in outputs[0]])

[Var(x_0, ty=TensorType([16, 3, 3, 3], float32)), Var(x_8, ty=TensorType([1, 32, 128, 128], float32)), Var(x_20, ty=TensorType([1, 96, 64, 64], float32))]


In [13]:
prev = subgraphs[0]

In [33]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']

In [11]:
tvm.IRModule.from_expr(run_opt_pass(subgraphs[0], transform.ToGraphNormalForm()))['main']


fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 256, 256), float32] */;
  %2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
  %3 = nn.conv2d(%2, meta[relay.Constant][2] /* ty=Tensor[(32, 16, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %4 = nn.bias_add(%3, meta[relay.Constant][3] /* ty=Tensor[(32), float32] */) /* ty=Tensor[(1, 32, 128, 128), float32] */;
  %5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 3

In [12]:
tvm.IRModule.from_expr(run_opt_pass(subgraphs[1], transform.ToGraphNormalForm()))['main']


fn (%x_20: Tensor[(1, 96, 64, 64), float32] /* ty=Tensor[(1, 96, 64, 64), float32] */, %x_8: Tensor[(1, 32, 128, 128), float32] /* ty=Tensor[(1, 32, 128, 128), float32] */) {
  %0 = nn.conv2d_transpose(%x_20, meta[relay.Constant][0] /* ty=Tensor[(96, 16, 3, 3), float32] */, channels=16, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 128, 128), float32] */;
  %2 = (%1, %x_8) /* ty=(Tensor[(1, 16, 128, 128), float32], Tensor[(1, 32, 128, 128), float32]) */;
  %3 = concatenate(%2, axis=1) /* ty=Tensor[(1, 48, 128, 128), float32] */;
  %4 = nn.conv2d_transpose(%3, meta[relay.Constant][2] /* ty=Tensor[(48, 1, 3, 3), float32] */, channels=1, kernel_size=[3, 3], strides=[2, 2], padding=[0i64, 0i64, 1i64, 1i64], kernel_layout="IOHW") /* ty=Tensor[(1, 1, 256, 256), float32] */;
  nn.bias_add(%4, meta[relay.C

In [28]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']
# outs = outputs[0][1]
outs = run_opt_pass(outputs[0][1], transform.ToGraphNormalForm())
outs = tvm.IRModule.from_expr(outs)['main']

# outnew = run_opt_pass(out_1, transform.ToGraphNormalForm())
# outnew = tvm.IRModule.from_expr(outnew)['main']

# out1 = relay.Function(params, relay.Tuple([outs, out]))

In [52]:
ann = run_opt_pass(subgraph, transform.ToGraphNormalForm())
submod = tvm.IRModule.from_expr(ann)['main']

In [44]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']
outnew = run_opt_pass(out_1, transform.ToGraphNormalForm())
outnew = tvm.IRModule.from_expr(outnew)['main']

out1 = relay.Function(out.params, relay.Tuple([outnew, out]), out.ret_type, out.type_params, out.attrs)

In [41]:
outputs[0][0].var.name_hint

'x_0'

In [33]:
out = relay.Tuple(outputs[0])
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']

[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_8 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_9 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_10 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_11 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_12 is bound more than once, this is not valid IR
[23:58:33] /home/jd/workspace/tvm-v0.9.0/src/relay/analysis/well_formed.cc:49: The IR is not well formed with: The variable x_1

TVMError: Traceback (most recent call last):
  6: TVMFuncCall
  5: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>::AssignTypedLambda<tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>(tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  4: tvm::IRModule

In [31]:
out = subgraphs[0]
ann = run_opt_pass(out, transform.ToGraphNormalForm())
out = tvm.IRModule.from_expr(ann)['main']
# outnew = [run_opt_pass(i, transform.ToGraphNormalForm()) for i in outputs[0]]
out1 = relay.Function(out.params, relay.Tuple(outnew), out.ret_type, out.type_params, out.attrs)
mod = tvm.IRModule()
mod["main"] = out1

# with tvm.transform.PassContext(opt_level=4):
#     lib = relay.build(out1, 'cuda', params=params)

TVMError: Traceback (most recent call last):
  3: TVMFuncCall
  2: tvm::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const [clone .isra.0]
  1: tvm::IRModuleNode::Add(tvm::GlobalVar const&, tvm::BaseFunc const&, bool)
  0: tvm::WarnIfMalformed(tvm::IRModule const&, tvm::relay::Function)
  File "/home/jd/workspace/tvm-v0.9.0/src/ir/module.cc", line 191
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: fv.size() == 0 (3 vs. 0) : Function:
fn (%input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */) {
  %0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3])

In [21]:

ann = run_opt_pass(subgraphs.body, transform.ToGraphNormalForm())
tmp_mod = tvm.IRModule.from_expr(ann)

[[LetNode(Var(x_225, ty=TensorType([1, 32, 128, 128], float32)), CallNode(Op(nn.bias_add), [Var(x_223, ty=TensorType([1, 32, 128, 128], float32)), Var(x_224, ty=TensorType([32], float32))], relay.attrs.BiasAddAttrs(0x30b172c8), [TensorType([1, 32, 128, 128], float32), TensorType([32], float32)]), LetNode(Var(x_226, ty=TensorType([1, 32, (int64)64, (int64)64], float32)), CallNode(Op(nn.max_pool2d), [Var(x_225, ty=TensorType([1, 32, 128, 128], float32))], relay.attrs.MaxPool2DAttrs(0x30aedf08), [TensorType([1, 32, 128, 128], float32)]), LetNode(Var(x_227, ty=TensorType([64, 32, 3, 3], float32)), Constant([[[[-0.00034386 -0.00349673  0.07773861]
     [-0.03190579 -0.01584832  0.02670189]
     [ 0.01921258 -0.07818282  0.03479443]]
  
    [[-0.00130421 -0.01830789  0.05642799]
     [-0.0170652  -0.02210277  0.04254849]
     [ 0.01875039 -0.08010064 -0.08040661]]
  
    [[ 0.02165145  0.0240651  -0.04487719]
     [-0.02167934 -0.00768209  0.0794398 ]
     [-0.02068903  0.06871933  0.0772698

In [16]:
mod['main'].params[0].name_hint

'x_20'

In [5]:
out = quantize(mod, 2)
dev = tvm.cuda(0)
target = 'cuda'
# with tvm.transform.PassContext(opt_level=4, disabled_pass={}):
#     lib = relay.build(out, target, params=params)

In [10]:
mod = tvm.IRModule()
mod['main'] = out

In [11]:
graph_attr = {"color": "red"}
node_attr = {"color": "blue"}
edge_attr = {"color": "black"}
def get_node_attr(node):
    if "nn.conv2d" in node.type_name and "NCHW" in node.detail:
        return {
            "fillcolor": "green",
            "style": "filled",
            "shape": "box",
        }
    if "Var" in node.type_name:
        return {"shape": "ellipse"}
    return {"shape": "box"}
    
dot_plotter = relay_viz.DotPlotter(
    graph_attr=graph_attr,
    node_attr=node_attr,
    edge_attr=edge_attr,
    get_node_attr=get_node_attr)

viz = relay_viz.RelayVisualizer(
    # submods[1],
    mod,
    relay_param=params,
    plotter=dot_plotter,
    parser=relay_viz.DotVizParser())
viz.render("newnew")

# viz = relay_viz.RelayVisualizer(
#     # submods[0],
#     out1,
#     relay_param=params,
#     plotter=dot_plotter,
#     parser=relay_viz.DotVizParser())
# viz.render("simple_0")
# viz = relay_viz.RelayVisualizer(
#     mod,
#     relay_param=params,
#     plotter=dot_plotter,
#     parser=relay_viz.DotVizParser())
# viz.render("simple_total")

In [43]:
print(mod['main'].body.astext(False))

#[version = "0.0.5"]
free_var %input_1: Tensor[(1, 3, 256, 256), float32] /* ty=Tensor[(1, 3, 256, 256), float32] */;
%0 = nn.conv2d(%input_1, meta[relay.Constant][0] /* ty=Tensor[(16, 3, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=16, kernel_size=[3, 3]) /* ty=Tensor[(1, 16, 256, 256), float32] */;
%1 = nn.bias_add(%0, meta[relay.Constant][1] /* ty=Tensor[(16), float32] */) /* ty=Tensor[(1, 16, 256, 256), float32] */;
%2 = nn.max_pool2d(%1, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=Tensor[(1, 16, 128i64, 128i64), float32] */;
%3 = nn.conv2d(%2, meta[relay.Constant][2] /* ty=Tensor[(32, 16, 3, 3), float32] */, padding=[1i64, 1i64, 1i64, 1i64], channels=32, kernel_size=[3, 3]) /* ty=Tensor[(1, 32, 128, 128), float32] */;
%4 = nn.bias_add(%3, meta[relay.Constant][3] /* ty=Tensor[(32), float32] */) /* ty=Tensor[(1, 32, 128, 128), float32] */;
%5 = nn.max_pool2d(%4, pool_size=[2, 2], strides=[2, 2], padding=[0i64, 0i64, 0i64, 0i64]) /* ty=

In [3]:
def get_network():
    img_size = 8
    out_channels = 16
    batch_size = 1
    data = relay.var("data", relay.TensorType((batch_size, 3, img_size, img_size), "float16"))
    dense_weight = relay.var(
        "dweight", relay.TensorType((batch_size, 16 * img_size * img_size), "float16")
    )
    weight = relay.var("weight")
    second_weight = relay.var("second_weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")
    simple_net = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.nn.batch_flatten(simple_net)
    simple_net = relay.nn.dense(simple_net, dense_weight)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)
    data_shape = (batch_size, 3, img_size, img_size)
    net, params = testing.create_workload(simple_net)
    return net, params, data_shape




In [19]:
import pygraphviz as pgv
import json
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            # if args.show_size == 1:
            if 1 == 1:
                src_size = 1
                for i in json_data['attrs']['shape'][1][src[0]]:
                    src_size = src_size * i
                
                dst_size = 1
                for i in json_data['attrs']['shape'][1][node_idx]:
                    dst_size = dst_size * i
                # print(src[0], json_data['nodes'][src[0]]['name'], src_size)

                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]) + "[{}]".format(src_size), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]) + "[{}]".format(dst_size))
            else:
                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')

In [63]:
show_graph(graph_json_raw, 'test')
